The complete design process for the compressor will encompass the following
steps:
1. choice of rotational speed and annulus dimensions;
2. determination of number of stages, using an assumed efficiency;
3. calculation of the air angles for each stage at the mean radius;
4. determination of the variation of the air angles from root to tip;
5. investigation of compressibility effects;
6. selection of compressor blading, using experimentally obtained cascade data;
7. check on efficiency previously assumed, using the cascade data;
8. estimation of off-design performance;
9. rig testing.


In [110]:
import numpy as np
from tabulate import tabulate
from scipy.optimize import fsolve

In [111]:
#Solver for a two system of equations of Beta 1 and Beta 2
def solve_system(beta1_minus_beta2, beta1_plus_beta2):
    # Define the system of equations
    def equations(vars):
        B1, B2 = vars
        eq1 = np.tan(B1) - np.tan(B2) - beta1_minus_beta2
        eq2 = np.tan(B1) + np.tan(B2) - beta1_plus_beta2
        return [eq1, eq2]

    # Initial guess for B1 and B2
    initial_guess = [0.1, 0.1]

    # Solve the system of equations
    solution = fsolve(equations, initial_guess)

    # Convert radians to degrees
    B1_degrees = np.degrees(solution[0])
    B2_degrees = np.degrees(solution[1])

    return B1_degrees, B2_degrees

In [112]:
#1. choice of rotational speed and annulus dimensions;

#Sea Level Conditions
Ta = 288                                   #ambient temperature        [K]
Pa = 101                                   #ambient pressure           [KPa]
Cp = 1.005                                 #specif heat of air         [kJ/kg*K]
gammaAir = 1.4                             #heat capacity ratio        [-]
R = 0.287                                  #gas constant of air        [kJ/kg*K]

#Specified Data
CPR =  5.22                                  #compressor pressure ratio  [-]
mdot = 10.3                                   #air mass flow              [kg/s]

#Previous Experience Data
Ut = 350                                    #tip speed                  [m/s]
Ca = 150                                   #axial velocity             [m/s]
hub_tip_range = np.arange(0.40, 0.60 + 0.05, 0.05)  #hub-to-tip ratio           [-]

#Assuming no loss in intake
T01 = Ta
P01 = Pa

#No pre-swirl (NO IGV)
Cw1 = 0
Ca1 = Ca
C1 = Ca1

#Inlet Static Conditions
T1 = T01 - ((C1**2)/(2*Cp*1E3))
P1 = P01*((T1)/(T01))**((gammaAir)/(gammaAir-1))
rho1 = (P1)/(R*T1)

#Tip radius
rt_range = np.sqrt((mdot)/(np.pi*rho1*Ca1*(1-(hub_tip_range)**2)))

#Angular velocity
N_range = (Ut)/(2*np.pi*rt_range)
RPM = N_range * 60  # Convert rev/s to RPM

# Combine the arrays into a list of lists
table_data = list(zip(hub_tip_range, rt_range, N_range, RPM))

#Print table
headers = ["hub_tip", "rt [m]", "N [rev/s]", "N [RPM]"]
print(tabulate(table_data, headers=headers, floatfmt=".4f", tablefmt="github"))

|   hub_tip |   rt [m] |   N [rev/s] |    N [RPM] |
|-----------|----------|-------------|------------|
|    0.4000 |   0.1533 |    363.2720 | 21796.3224 |
|    0.4500 |   0.1574 |    353.9628 | 21237.7707 |
|    0.5000 |   0.1623 |    343.2598 | 20595.5888 |
|    0.5500 |   0.1683 |    331.0278 | 19861.6666 |
|    0.6000 |   0.1757 |    317.0898 | 19025.3900 |


In [113]:
N_adjusted = 20600/60
hub_tip = 0.5
rt_inlet = 0.1623

#Tip speed
Ut = 2*np.pi*rt_inlet*N_adjusted

U1t = Ut
V1t = np.sqrt(U1t**2+Ca1**2)

#Speed of Sound
a = np.sqrt(gammaAir*R*1E3*T1)

M1t = V1t/a

rh_inlet = round(hub_tip*rt_inlet,4)

rm = (rh_inlet+rt_inlet)/2

#Compressor Exit properties
P02 = CPR*P01

e_pol = 0.9
n_1_n = (1/e_pol)*((gammaAir-1)/(gammaAir))

T02 = T01*(P02/P01)**(n_1_n)

Ca2 = Ca1
T2 = T02 - ((Ca2**2)/(2*Cp*1E3))
P2 = P02*(T2/T02)**((gammaAir)/(gammaAir-1))
rho2 = (P2)/(R*T2)

#Exit Area
A2 = mdot/(rho2*Ca2)

#Blade Height
h = A2/(2*np.pi*rm)

rt_exit = rm + (h/2)
rh_exit = rm - (h/2)

#Temperature increase for the complete compressor
deltaTComp = T02-T01

U = 2*np.pi*rm*N_adjusted

print(round(U,1))

beta1 = np.degrees(np.arctan(U/Ca))

V1 = (Ca)/(np.cos(np.radians(beta1)))

#de Haller Criterior
deHallerNumber = 0.72
V2 = V1*deHallerNumber

beta2 = np.degrees(np.arccos(Ca/V2))

#Temperature increse in each stage
deltaTStage = (U*Ca*(np.tan(np.radians(beta1))-np.tan(np.radians(beta2))))/(Cp*1E3)


#Number of stages
Nstages_t = int(np.ceil(deltaTComp/deltaTStage)) + 1

print(round(rh_exit,4))

262.5
0.1091


In [114]:
#Work done factor
wdf = np.full(Nstages_t,0.83)
wdf[0] = 0.98
wdf[1] = 0.93
wdf[2] = 0.88

deltaT0S = np.full(Nstages_t,25)
deltaT0S[0] = 20    #For first stage

#Save data to an array for each stage calculations
StageNumber = np.arange(1,Nstages_t + 1)
beta1_stages = np.zeros(Nstages_t)
beta2_stages = np.zeros(Nstages_t)
alpha1_stages = np.zeros(Nstages_t)
alpha2_stages = np.zeros(Nstages_t)
alpha3_stages = np.zeros(Nstages_t)
deltaBeta_stages = np.zeros(Nstages_t)
Cw1_stages = np.zeros(Nstages_t)
Cw2_stages = np.zeros(Nstages_t)
deltaCw_stages = np.zeros(Nstages_t)
P03_P01_stages = np.zeros(Nstages_t)
P03_stages = np.zeros(Nstages_t)
T03_stages = np.zeros(Nstages_t)
deHallerCriterion_stagesR = np.zeros(Nstages_t)
deHallerCriterion_stagesS = np.zeros(Nstages_t)


#Iterate in each stage and solve the system
for i in StageNumber:
    if(i==1 and i != Nstages_t):
        print("Stage: ",i)
        deltaCw_1 = (Cp*1E3*deltaT0S[i-1])/(wdf[i-1]*U)
        deltaCw_stages[i-1] = deltaCw_1

        Cw1_1 = 0
        Cw2_1 = Cw1_1 + deltaCw_1

        Cw1_stages[i-1] = Cw1_1
        Cw2_stages[i-1] = Cw2_1

        beta1_1 = np.degrees(np.arctan(U/Ca))
        beta2_1 = np.degrees(np.arctan((U-Cw2_1)/Ca))
        alpha1_1 = 0  #No IGV
        alpha2_1 = np.degrees(np.arctan((Cw2_1)/Ca))

        alpha1_stages[i-1] = alpha1_1
        alpha2_stages[i-1] = alpha2_1
        beta1_stages[i-1] = beta1_1
        beta2_stages[i-1] = beta2_1

        #Deflection in the rotor blades
        deltaBeta_1 = beta1_1 - beta2_1
        deltaBeta_stages[i-1] = deltaBeta_1

        deHallerCriterion = np.cos(np.radians(beta1_1))/np.cos(np.radians(beta2_1))
        if(deHallerCriterion >= deHallerNumber): print("DeHaller Valid [Rotor]: ", deHallerCriterion)
        else: print("DeHaller Invalid [Rotor]: ", deHallerCriterion)
        deHallerCriterion_stagesR[i-1] = deHallerCriterion

        P03_P01_1 = (1+((e_pol*deltaT0S[i-1])/(T01)))**((gammaAir)/(gammaAir-1))

        P03_1 = P01*P03_P01_1
        T03_1 = T01 + deltaT0S[i-1]

        P03_P01_stages[i-1] = P03_P01_1
        P03_stages[i-1] = P03_1
        T03_stages[i-1] = T03_1

        #Degree of Reaction
        Λ_1 =  1 - ((Cw2_1+Cw1_1)/(2*U))

    elif(i==2 and i != Nstages_t):
        print("Stage: ",i)
        Λ_2 = 0.70
        deltaT0S[i-1] = 23
        tanB1_minus_tanB2 = (deltaT0S[i-1]*Cp*1E3)/(wdf[i-1]*U*Ca)
        tanB1_plus_tanB2 =  (Λ_2*2*U)/(Ca)

        beta1_2, beta2_2 = solve_system(tanB1_minus_tanB2, tanB1_plus_tanB2)

        alpha1_2 = np.degrees(np.arctan((U/Ca)-np.tan(np.radians(beta1_2))))
        alpha2_2 = np.degrees(np.arctan((U/Ca)-np.tan(np.radians(beta2_2))))

        alpha1_stages[i-1] = alpha1_2
        alpha2_stages[i-1] = alpha2_2
        beta1_stages[i-1] = beta1_2
        beta2_stages[i-1] = beta2_2

        Cw1_2 = Ca*np.tan(np.radians(alpha1_2))
        Cw2_2 = Ca*np.tan(np.radians(alpha2_2))

        deltaCw_2 = Cw2_2 - Cw1_2

        Cw1_stages[i-1] = Cw1_2
        Cw2_stages[i-1] = Cw2_2
        deltaCw_stages[i-1] = deltaCw_2

        #Deflection in the rotor blades
        deltaBeta_2 = beta1_2 - beta2_2
        deltaBeta_stages[i-1] = deltaBeta_2

        alpha3_1 = alpha1_2
        alpha3_stages[i-2] = alpha3_1

        deHallerCriterion = np.cos(np.radians(beta1_2))/np.cos(np.radians(beta2_2))
        if(deHallerCriterion >= deHallerNumber): print("DeHaller Valid [Rotor]: ", deHallerCriterion)
        else: print("DeHaller Invalid [Rotor]: ", deHallerCriterion)
        deHallerCriterion_stagesR[i-1] = deHallerCriterion

        deHallerCriterion = np.cos(np.radians(alpha2_2))/np.cos(np.radians(alpha3_1))
        if(deHallerCriterion >= deHallerNumber): print("DeHaller Valid [Stator]: ", deHallerCriterion)
        else: print("DeHaller Invalid [Stator]: ", deHallerCriterion)
        deHallerCriterion_stagesS[i-1] = deHallerCriterion

        T01_2 = T03_stages[i-2]
        P01_2 = P03_stages[i-2]

        P03_P01_2 = (1+((e_pol*deltaT0S[i-1])/(T01_2)))**((gammaAir)/(gammaAir-1))

        P03_2 = P01_2*P03_P01_2
        T03_2 = T01_2 + deltaT0S[i-1]

        P03_P01_stages[i-1] = P03_P01_2
        P03_stages[i-1] = P03_2
        T03_stages[i-1] = T03_2

    elif(i==3 and i != Nstages_t):
        print("Stage: ",i)
        Λ_3 = 0.50
        deltaT0S[i-1] = 23

        tanB1_minus_tanB2 = (deltaT0S[i-1]*Cp*1E3)/(wdf[i-1]*U*Ca)
        tanB1_plus_tanB2 =  (Λ_3*2*U)/(Ca)

        beta1_3, beta2_3 = solve_system(tanB1_minus_tanB2, tanB1_plus_tanB2)
        alpha1_3 = beta2_3
        alpha2_3 = beta1_3

        alpha1_stages[i-1] = alpha1_3
        alpha2_stages[i-1] = alpha2_3
        beta1_stages[i-1] = beta1_3
        beta2_stages[i-1] = beta2_3

        Cw1_3 = Ca*np.tan(np.radians(alpha1_3))
        Cw2_3 = Ca*np.tan(np.radians(alpha2_3))

        deltaCw_3 = Cw2_3 - Cw1_3

        Cw1_stages[i-1] = Cw1_3
        Cw2_stages[i-1] = Cw2_3
        deltaCw_stages[i-1] = deltaCw_3

        #Deflection in the rotor blades
        deltaBeta_3 = beta1_3 - beta2_3
        deltaBeta_stages[i-1] = deltaBeta_3

        alpha3_2 = alpha1_3
        alpha3_stages[i-2] = alpha3_2

        deHallerCriterion = np.cos(np.radians(beta1_3))/np.cos(np.radians(beta2_3))
        if(deHallerCriterion >= deHallerNumber): print("DeHaller Valid [Rotor]: ", deHallerCriterion)
        else: print("DeHaller Invalid [Rotor]: ", deHallerCriterion)
        deHallerCriterion_stagesR[i-1] = deHallerCriterion

        deHallerCriterion = np.cos(np.radians(alpha2_3))/np.cos(np.radians(alpha3_2))
        if(deHallerCriterion >= deHallerNumber): print("DeHaller Valid [Stator]: ", deHallerCriterion)
        else: print("DeHaller Invalid [Stator]: ", deHallerCriterion)
        deHallerCriterion_stagesS[i-1] = deHallerCriterion

        T01_3 = T03_stages[i-2]
        P01_3 = P03_stages[i-2]

        P03_P01_3 = (1+((e_pol*deltaT0S[i-1])/(T01_3)))**((gammaAir)/(gammaAir-1))

        P03_3 = P01_3*P03_P01_3
        T03_3 = T01_3 + deltaT0S[i-1]

        P03_P01_stages[i-1] = P03_P01_3
        P03_stages[i-1] = P03_3
        T03_stages[i-1] = T03_3

    elif(i>=4 and i!=Nstages_t):
        print("Stage: ",i)
        Λ_s = 0.50
        deltaT0S[i-1] = 22
        tanB1_minus_tanB2 = (deltaT0S[i-1]*Cp*1E3)/(wdf[i-1]*U*Ca)
        tanB1_plus_tanB2 =  (Λ_s*2*U)/(Ca)

        beta1_s, beta2_s = solve_system(tanB1_minus_tanB2, tanB1_plus_tanB2)
        alpha1_s = beta2_s
        alpha2_s = beta1_s

        alpha1_stages[i-1] = alpha1_s
        alpha2_stages[i-1] = alpha2_s
        beta1_stages[i-1] = beta1_s
        beta2_stages[i-1] = beta2_s

        Cw1_s = Ca*np.tan(np.radians(alpha1_s))
        Cw2_s = Ca*np.tan(np.radians(alpha2_s))

        deltaCw_s = Cw2_s - Cw1_s

        Cw1_stages[i-1] = Cw1_s
        Cw2_stages[i-1] = Cw2_s
        deltaCw_stages[i-1] = deltaCw_s

        #Deflection in the rotor blades
        deltaBeta_s = beta1_s - beta2_s
        deltaBeta_stages[i-1] = deltaBeta_s

        alpha3_3 = alpha1_s
        alpha3_stages[i-2] = alpha3_3

        deHallerCriterion = np.cos(np.radians(beta1_s))/np.cos(np.radians(beta2_s))
        if(deHallerCriterion >= deHallerNumber): print("DeHaller Valid [Rotor]: ", deHallerCriterion)
        else: print("DeHaller Invalid [Rotor]: ", deHallerCriterion)
        deHallerCriterion_stagesR[i-1] = deHallerCriterion

        deHallerCriterion = np.cos(np.radians(alpha2_s))/np.cos(np.radians(alpha3_3))
        if(deHallerCriterion >= deHallerNumber): print("DeHaller Valid [Stator]: ", deHallerCriterion)
        else: print("DeHaller Invalid [Stator]: ", deHallerCriterion)
        deHallerCriterion_stagesS[i-1] = deHallerCriterion

        T01_s = T03_stages[i-2]
        P01_s = P03_stages[i-2]

        P03_P01_s = (1+((e_pol*deltaT0S[i-1])/(T01_s)))**((gammaAir)/(gammaAir-1))

        P03_s = P01_s*P03_P01_s
        T03_s = T01_s + deltaT0S[i-1]

        P03_P01_stages[i-1] = P03_P01_s
        P03_stages[i-1] = P03_s
        T03_stages[i-1] = T03_s
    
    elif(i==Nstages_t):
        print("Stage: ",i)
        Λ_f = 0.50

        T01_f = T03_stages[i-2]
        P01_f = P03_stages[i-2]
        P03_P01_f = (CPR*P01)/P01_f

        deltaT0S_f = ((T01_f)/(e_pol))*(((P03_P01_f)**((gammaAir-1)/(gammaAir)))-1) 
        deltaT0S[i-1] = deltaT0S_f
        tanB1_minus_tanB2 = (deltaT0S[i-1]*Cp*1E3)/(wdf[i-1]*U*Ca)
        tanB1_plus_tanB2 =  (Λ_f*2*U)/(Ca)

        beta1_f, beta2_f = solve_system(tanB1_minus_tanB2, tanB1_plus_tanB2)
        alpha1_f = beta2_f
        alpha2_f = beta1_f

        alpha1_stages[i-1] = alpha1_f
        alpha2_stages[i-1] = alpha2_f
        beta1_stages[i-1] = beta1_f
        beta2_stages[i-1] = beta2_f

        Cw1_f = Ca*np.tan(np.radians(alpha1_f))
        Cw2_f = Ca*np.tan(np.radians(alpha2_f))

        deltaCw_f = Cw2_f - Cw1_f

        Cw1_stages[i-1] = Cw1_f
        Cw2_stages[i-1] = Cw2_f
        deltaCw_stages[i-1] = deltaCw_f

        #Deflection in the rotor blades
        deltaBeta_f = beta1_f - beta2_f
        deltaBeta_stages[i-1] = deltaBeta_f

        alpha3_f = alpha1_f
        alpha3_stages[i-2] = alpha3_f

        deHallerCriterion = np.cos(np.radians(beta1_f))/np.cos(np.radians(beta2_f))
        if(deHallerCriterion >= deHallerNumber): print("DeHaller Valid [Rotor]: ", deHallerCriterion)
        else: print("DeHaller Invalid [Rotor]: ", deHallerCriterion)
        deHallerCriterion_stagesR[i-1] = deHallerCriterion

        deHallerCriterion = np.cos(np.radians(alpha2_f))/np.cos(np.radians(alpha3_f))
        if(deHallerCriterion >= deHallerNumber): print("DeHaller Valid [Stator]: ", deHallerCriterion)
        else: print("DeHaller Invalid [Stator]: ", deHallerCriterion)
        deHallerCriterion_stagesS[i-1] = deHallerCriterion

        P03_f = P01_f*P03_P01_f
        T03_f = T01_f + deltaT0S[i-1]

        P03_P01_stages[i-1] = P03_P01_f
        P03_stages[i-1] = P03_f
        T03_stages[i-1] = T03_f

    else:
        print("Invalid Process")

Stage:  1
DeHaller Valid [Rotor]:  0.7861787127069165
Stage:  2
DeHaller Valid [Rotor]:  0.7359465335559903
DeHaller Valid [Stator]:  0.7820785875497135
Stage:  3
DeHaller Valid [Rotor]:  0.7249722578975776
DeHaller Valid [Stator]:  0.7249722578975776
Stage:  4
DeHaller Valid [Rotor]:  0.7218713192813131
DeHaller Valid [Stator]:  0.7218713192813131
Stage:  5
DeHaller Valid [Rotor]:  0.7218713192813131
DeHaller Valid [Stator]:  0.7218713192813131
Stage:  6
DeHaller Valid [Rotor]:  0.7218713192813131
DeHaller Valid [Stator]:  0.7218713192813131
Stage:  7
DeHaller Valid [Rotor]:  0.7218713192813131
DeHaller Valid [Stator]:  0.7218713192813131
Stage:  8
DeHaller Valid [Rotor]:  0.7218713192813131
DeHaller Valid [Stator]:  0.7218713192813131
Stage:  9
DeHaller Valid [Rotor]:  0.7320569205218961
DeHaller Valid [Stator]:  0.7320569205218961


In [115]:
rh_in_rotor_stages = np.zeros(Nstages_t)
rm_stages = np.zeros(Nstages_t)
rt_in_rotor_stages = np.zeros(Nstages_t)

Ur_stages = np.zeros(Nstages_t)
Um_stages = np.zeros(Nstages_t)
Ut_stages = np.zeros(Nstages_t)

beta1r_stages = np.zeros(Nstages_t)
beta1m_stages = np.zeros(Nstages_t)
beta1t_stages = np.zeros(Nstages_t)

C3_stages = np.zeros(Nstages_t)
T3_stages = np.zeros(Nstages_t)
P3_stages = np.zeros(Nstages_t)
rho3_stages = np.zeros(Nstages_t)
A3_stages = np.zeros(Nstages_t)
h_stages = np.zeros(Nstages_t)

rt_out_stator_stages = np.zeros(Nstages_t)
rh_out_stator_stages = np.zeros(Nstages_t)

rt_out_rotor_stages = np.zeros(Nstages_t)
rh_out_rotor_stages = np.zeros(Nstages_t)

Ut_out_rotor_stages = np.zeros(Nstages_t)
Ur_out_rotor_stages = np.zeros(Nstages_t)

Cw2m_stages = Cw2_stages
Cw2r_stages = np.zeros(Nstages_t)
Cw2t_stages = np.zeros(Nstages_t)

alpha2r_stages = np.zeros(Nstages_t)
alpha2m_stages = np.zeros(Nstages_t)
alpha2t_stages = np.zeros(Nstages_t)

beta2r_stages = np.zeros(Nstages_t)
beta2m_stages = np.zeros(Nstages_t)
beta2t_stages = np.zeros(Nstages_t)

deltaBetar_stages = np.zeros(Nstages_t)
deltaBetam_stages = deltaBeta_stages
deltaBetat_stages = np.zeros(Nstages_t)

DegR_r_stages = np.zeros(Nstages_t)

for i in range(Nstages_t):
    if(i==0):
        rh_in_rotor_stages[i] = rh_inlet
        rt_in_rotor_stages[i] = rt_inlet
    
    else:
        rh_in_rotor_stages[i] = rh_out_stator_stages[i-1]
        rt_in_rotor_stages[i] = rt_out_stator_stages[i-1]

    rm_stages[i] = (rh_in_rotor_stages[i] + rt_in_rotor_stages[i])/2

    Ur_stages[i] = 2*np.pi*rh_in_rotor_stages[i]*N_adjusted
    Um_stages[i] = 2*np.pi*rm_stages[i]*N_adjusted
    Ut_stages[i] = 2*np.pi*rt_in_rotor_stages[i]*N_adjusted

    beta1r_stages[i] = np.degrees(np.arctan((Ur_stages[i])/(Ca)))
    beta1m_stages[i] = np.degrees(np.arctan((Um_stages[i])/(Ca)))
    beta1t_stages[i] = np.degrees(np.arctan((Ut_stages[i])/(Ca)))

    C3_stages[i] = (Ca)/(np.cos(np.radians(alpha3_stages[i])))
    T3_stages[i] = T03_stages[i] - ((C3_stages[i]**2)/(2*Cp*1E3))
    P3_stages[i] = P03_stages[i]*((T3_stages[i])/(T03_stages[i]))**((gammaAir)/(gammaAir-1))
    rho3_stages[i] = (P3_stages[i])/(R*T3_stages[i])
    A3_stages[i] = (mdot)/(rho3_stages[i]*Ca)
    h_stages[i] = (A3_stages[i])/(2*np.pi*rm_stages[i])
    rt_out_stator_stages[i] = rm_stages[i] + (h_stages[i]/2)
    rh_out_stator_stages[i] = rm_stages[i] - (h_stages[i]/2)

    rt_out_rotor_stages[i] =(rt_in_rotor_stages[i] +  rt_out_stator_stages[i])/2
    rh_out_rotor_stages[i] =(rh_in_rotor_stages[i] +  rh_out_stator_stages[i])/2

    Ut_out_rotor_stages[i] = 2*np.pi*rt_out_rotor_stages[i]*N_adjusted
    Ur_out_rotor_stages[i] = 2*np.pi*rh_out_rotor_stages[i]*N_adjusted

    Cw2r_stages[i] = deltaCw_stages[i]*(rm_stages[i]/rh_out_rotor_stages[i])
    Cw2t_stages[i] = deltaCw_stages[i]*(rm_stages[i]/rt_out_rotor_stages[i])

    alpha2r_stages[i] = np.degrees(np.arctan(Cw2r_stages[i]/Ca))
    alpha2m_stages[i] = np.degrees(np.arctan(Cw2m_stages[i]/Ca))
    alpha2t_stages[i] = np.degrees(np.arctan(Cw2t_stages[i]/Ca))

    beta2r_stages[i] = np.degrees(np.arctan((Ur_out_rotor_stages[i] - Cw2r_stages[i])/Ca))
    beta2m_stages[i] = np.degrees(np.arctan((Um_stages[i] - Cw2m_stages[i])/Ca))
    beta2t_stages[i] = np.degrees(np.arctan((Ut_out_rotor_stages[i] - Cw2t_stages[i])/Ca))

    deltaBetar_stages[i] = beta1r_stages[i] - beta2r_stages[i]
    deltaBetat_stages[i] = beta1t_stages[i] - beta2t_stages[i]

    DegR_r_stages[i] = 1 - ((Cw2r_stages[i])/(2*Ur_out_rotor_stages[i]))

In [122]:
import csv
from tabulate import tabulate

# Define the filename
filename = "compressor_design_data.csv"

# Define a function to round numbers to 4 decimal places
def round_to_4_digits(value):
    try:
        return "{:.4f}".format(float(value))
    except (ValueError, TypeError):
        return value

# Open the file in write mode
with open(filename, "w", newline="") as file:
    # Create a CSV writer object
    writer = csv.writer(file)

    # Write specified data
    writer.writerow(["Design Point"])
    writer.writerow(["Compressor Pressure Ratio", round_to_4_digits(CPR)])
    writer.writerow(["Air Mass Flow (kg/s)", round_to_4_digits(mdot)])
    writer.writerow([])  # Empty row as separator

    # Write ambient conditions
    writer.writerow(["Ambient Conditions [Take-Off]"])
    writer.writerow(["Ambient Temperature (K)", round_to_4_digits(Ta)])
    writer.writerow(["Ambient Pressure (KPa)", round_to_4_digits(Pa)])
    writer.writerow(["Specific Heat of Air (kJ/kg*K)", round_to_4_digits(Cp)])
    writer.writerow(["Heat Capacity Ratio", round_to_4_digits(gammaAir)])
    writer.writerow(["Gas Constant of Air (kJ/kg*K)", round_to_4_digits(R)])
    writer.writerow([])  # Empty row as separator

    # Write inlet conditions
    writer.writerow(["Inlet Static Conditions"])
    writer.writerow(["Inlet Temperature (K)", round_to_4_digits(T1)])
    writer.writerow(["Inlet Pressure (KPa)", round_to_4_digits(P1)])
    writer.writerow(["Inlet Density (kg/m^3)", round_to_4_digits(rho1)])
    writer.writerow(["Inlet Velocity (m/s)", round_to_4_digits(Ca1)])
    writer.writerow([])  # Empty row as separator

    # Write outlet conditions
    writer.writerow(["Outlet Static Conditions"])
    writer.writerow(["Outlet Temperature (K)", round_to_4_digits(T2)])
    writer.writerow(["Outlet Pressure (KPa)", round_to_4_digits(P2)])
    writer.writerow(["Outlet Density (kg/m^3)", round_to_4_digits(rho2)])
    writer.writerow(["Outlet Velocity (m/s)", round_to_4_digits(Ca2)])
    writer.writerow([])  # Empty row as separator

    # Write table data
    writer.writerow(["Axial Flow Compressor Preliminary Design Data"])
    headers = ["hub_tip", "rt [m]", "N [rev/s]", "N [RPM]"]
    writer.writerow(headers)
    for data in table_data:
        writer.writerow([round_to_4_digits(value) for value in data])
    writer.writerow([])  # Empty row as separator

    # Write header for stage data
    writer.writerow(["Stage", "Beta1 (deg)", "Beta2 (deg)", "Alpha1 (deg)", "Alpha2 (deg)", 
                     "Alpha3 (deg)", "Delta Beta (deg)", "Cw1 (m/s)", "Cw2 (m/s)", 
                     "Delta Cw (m/s)", "Work-done Factor", "DeltaT0S", "P03/P01", 
                     "P03 (KPa)", "T03 (K)", "Degree of Reaction", 
                     "DeHaller Criterion (Rotor)", "DeHaller Criterion (Stator)"])

    # Write data for each stage
    for i in range(Nstages_t):
        # Add your existing data
        writer.writerow([
            i+1, 
            *map(round_to_4_digits, [beta1_stages[i], beta2_stages[i], alpha1_stages[i], alpha2_stages[i], alpha3_stages[i], deltaBeta_stages[i], Cw1_stages[i], Cw2_stages[i], deltaCw_stages[i], wdf[i], deltaT0S[i], P03_P01_stages[i], P03_stages[i], T03_stages[i], locals().get(f'Λ_{i+1}', ''), deHallerCriterion_stagesR[i], deHallerCriterion_stagesR[i]])
        ])

    # Add new values for Radius at each stage
    writer.writerow([])  # Empty row as separator
    writer.writerow(["Stage Blade Deflection Calculation (FVM)"])
    writer.writerow(["Geometry Calculations"])
    writer.writerow(["Stage", "Constant Mean radius (m)", "Rotor inlet hub radius (m)", "Rotor inlet tip radius (m)",
                     "Rotor outlet hub radius (m)", "Rotor outlet tip radius (m)",
                     "Stator outlet hub radius (m)", "Stator outlet tip radius (m)",
                     "Exit Area (m^2)", "Stator Blade Height (m)"
                     
        ])
    
    # Write data for Radius at each stage
    for i in range(Nstages_t):
        # Add your existing data
        writer.writerow([
            i+1, 
            *map(round_to_4_digits, [rm_stages[i], rh_in_rotor_stages[i], rt_in_rotor_stages[i], 
                                     rh_out_rotor_stages[i],rt_out_rotor_stages[i],
                                     rh_out_stator_stages[i],rt_out_stator_stages[i],
                                     A3_stages[i],h_stages[i]])
        ])

    # Add new values for blade speed at each stage
    writer.writerow([])  # Empty row as separator
    writer.writerow(["Blade Speed Calculations"])
    writer.writerow(["Stage", "Mean blade speed (m/s)", "Rotor inlet hub blade speed (m/s)", "Rotor inlet tip blade speed (m/s)",
                     "Rotor outlet hub blade speed (m/s)", "Rotor outlet tip blade speed (m/s)",
                     "Rotor outlet mean relative speed (m/s)", "Rotor outlet hub relative speed (m/s)", "Rotor outlet tip relative speed (m/s)",
                     "RStage outlet speed [C3] (m/s)"
                     ])
    
    # Write data for blade speed at each stage
    for i in range(Nstages_t):
        # Add your existing data
        writer.writerow([
            i+1, 
            *map(round_to_4_digits, [Um_stages[i], Ur_stages[i],Ut_stages[i],
                                     Ur_out_rotor_stages[i],Ut_out_rotor_stages[i],
                                     Cw2m_stages[i],Cw2r_stages[i],Cw2t_stages[i],
                                     C3_stages[i]])
        ])

    # Add new values for blade speed at each stage
    writer.writerow([])  # Empty row as separator
    writer.writerow(["Angles Calculations"])
    writer.writerow(["Stage", "Beta1 mean (deg)", "Beta1 hub (deg)", "Beta1 tip (deg)",
                     "Beta2 mean (deg)", "Beta2 hub (deg)", "Beta2 tip (deg)",
                     "Mean Blade Deflection (deg)", "Hub Blade Deflection (deg)", "Tip Blade Deflection (deg)",
                     "Alpha2 mean (deg)", "Alpha2 hub (deg)", "Alpha2 tip (deg)"
                     ])
    
    # Write data for blade speed at each stage
    for i in range(Nstages_t):
        # Add your existing data
        writer.writerow([
            i+1, 
            *map(round_to_4_digits, [beta1m_stages[i],beta1r_stages[i], beta1t_stages[i],
                                     beta2m_stages[i],beta2r_stages[i], beta2t_stages[i],
                                     deltaBetam_stages[i],deltaBetar_stages[i], deltaBetat_stages[i],
                                     alpha2m_stages[i],alpha2r_stages[i], alpha2t_stages[i]])
        ])
    
    # Add new values for thermodynamic conditions at each stage
    writer.writerow([])  # Empty row as separator
    writer.writerow(["Thermodynamics Exit Calculations"])
    writer.writerow(["Stage", "Total Exit Temperature (K)","Static Exit Temperature (K)",
                     "Total Exit Pressure (kPa)","Static Exit Pressure (kPa)",
                     "Exit Density (kg/m^3)",
                     "Hub Degree of Reaction"
                     ])
    
    # Write data for thermodynamic conditions at each stage
    for i in range(Nstages_t):
        # Add your existing data
        writer.writerow([
            i+1, 
            *map(round_to_4_digits, [T03_stages[i],T3_stages[i],
                                     P03_stages[i],P3_stages[i],
                                     rho3_stages[i],
                                     DegR_r_stages[i]])
        ])
print("Data saved to", filename)

Data saved to compressor_design_data.csv
